2018-12-27 17:01:51 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st

In [ ]:
import os

plt.style.use(os.path.join(os.getcwd(), 'mystyle.mplstyle'))

# Get the data

In [ ]:
import urllib.request
suicide_rate_url = 'http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MH_12&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;SEX'
local_filename, headers = urllib.request.urlretrieve(suicide_rate_url, filename='who_suicide_rates.csv')
#local_filename, headers = urllib.request.urlretrieve(sucide_rate_url)

In [ ]:
local_filename = 'who_suicide_rates.csv'
import pandas 
from pandas_datareader import data
rates = pandas.read_csv(local_filename, names=['Country','Both', 'Female', 'Male'], skiprows=3)
rates.head(10)

In [ ]:
rates.plot.hist(stacked=True,y=['Male', 'Female'],
                bins=30, color=['Coral', 'Green'])
plt.xlabel('Rate');

In [ ]:
print(rates['Male'].mean(), rates['Female'].mean())

In [ ]:
rates.boxplot(['Both', 'Male','Female']);

In [ ]:
print(rates[rates['Both']>40])

In [ ]:
def plot_cdf(data, plot_range=None, scale_to=None, nbins=False, **kwargs):
    if not nbins:
        nbins= len(data)
    sorted_data = np.array(sorted(data), dtype=np.float64)
    data_range = sorted_data[-1] - sorted_data[0]
    counts, bin_edges = np.histogram(sorted_data, bins=nbins)
    xvalues = bin_edges[1:]
    yvalues = np.cumsum(counts)
    if plot_range is None:
        xmin = xvalues[0]
        xmax = xvalues[-1]
    else:
        xmin, xmax = plot_range
    # pad the arrays
    xvalues = np.concatenate([[xmin, xvalues[0]], xvalues, [xmax]])
    yvalues = np.concatenate([[0.0, 0.0],        yvalues, [yvalues.max()]])
    if scale_to:
        yvalues = yvalues / len(data) * scale_to
    plt.axis([xmin, xmax, 0, yvalues.max()])
    return plt.step(xvalues, yvalues, **kwargs)

In [ ]:
plot_cdf(rates['Both'], nbins=50, plot_range=[-5, 70])

In [ ]:
st.probplot(rates['Both'], dist='norm', plot=plt);

In [ ]:
eta = 1.
beta = 1.5
rvweib = st.weibull_min(beta, scale=eta)
results = st.probplot(rates['Both'], dist=rvweib, plot=plt)

In [ ]:
st.weibull_min.fit?

In [ ]:
beta, loc, eta = st.weibull_min.fit(rates['Both'], floc=0, scale = 12)
print(beta, loc, eta)

In [ ]:
rates['Both'].hist(bins=30)
np.random.seed(1100)
rvweib = st.weibull_min(beta, scale=eta)
plt.hist(rvweib.rvs(size=len(rates.Both)),bins=30, alpha=0.5);

In [ ]:
plot_cdf(rates['Both'], nbins=50,scale_to=1)
np.random.seed(1100)
plot_cdf(rvweib.rvs(size=50),scale_to=1)
plt.xlim((-2,50));

In [ ]:
coords = pandas.read_csv('data/country_centroids/country_centroids_primary.csv', sep='\t')
coords.keys()

In [ ]:
coords.head()

In [ ]:
rates['Lat'] = ''
rates['Lon'] = ''
for i in coords.index:
    ind = rates.Country.isin([coords.SHORT_NAME[i]])
    val = coords.loc[i, ['LAT', 'LONG']].values.astype('float')
    rates.loc[ind, ['Lat', 'Lon'] ] = list(val)

In [ ]:
rates.head()

In [ ]:
rates.loc[rates.Lat.isin(['']), ['Lat']] = np.nan
rates.loc[rates.Lon.isin(['']), ['Lon']] = np.nan
rates[['Lat', 'Lon']] = rates[['Lat', 'Lon']].astype('float')

In [ ]:
rates['DFE'] = ''
rates['DFE'] = abs(rates.Lat)
rates['DFE'] = rates['DFE'].astype('float')

In [ ]:
import matplotlib.patches as patches
import matplotlib.transforms as transforms

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(rates.DFE, rates.Both, '.')

trans = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)

rect = patches.Rectangle((0,0), width=23.5, height=1,
                         transform=trans, color='yellow',
                         alpha=0.5)
ax.set_xlabel('DFE')
ax.set_ylabel('Both')
ax.add_patch(rect);

In [ ]:
rates.DFE.hist(bins=13)
plt.xlabel('DFE')
plt.ylabel('Counts');

In [ ]:
bins = np.arange(23.5, 65+1,10, dtype='float')
bins = np.linspace(23.5, 65,11, dtype='float')
# now group the data into the bins
groups_rates = rates.groupby(np.digitize(rates.DFE, bins))

In [ ]:
import matplotlib.patches as patches
import matplotlib.transforms as transforms

fig = plt.figure()
ax = fig.add_subplot(111)
ax.errorbar(groups_rates.mean().DFE, 
            groups_rates.mean().Both, 
            yerr=np.array(groups_rates.std().Both),
            marker='.', 
            ls='None',
            lw=1.5,
            color='Green', 
            ms=1)
ax.plot(rates.DFE, rates.Both, '.', color='SteelBlue', ms=6)
trans = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)

rect = patches.Rectangle((0,0), width=23.5, height=1,
                         transform=trans, color='Yellow',
                         alpha=0.5)
ax.add_patch(rect)
ax.set_xlabel('DFE')
ax.set_ylabel('Both');

# Linear regression

In [ ]:
from scipy.stats import linregress
mindfe = 30.
selection = ~rates.DFE.isnull() * rates.DFE>mindfe
rv = rates[selection].as_matrix(columns=['DFE','Both'])
a, b, r, p, stderr = linregress(rv.T)
print('slope:{0:.4f}\nintercept:{1:.4f}\nrvalue:{2:.4f}\npvalue:{3:.4f}\nstderr:{4:.4f}'.format(a, b, r, p, stderr))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
rates.plot(kind='scatter', x='DFE', y='Both', ax=ax)
xdata = rates['DFE']
xmin, xmax = min(xdata), max(xdata)
xvalues = np.linspace(mindfe, xmax, 200)
yvalues = a * xvalues + b
ax.plot(xvalues, yvalues, color='red', lw=1.5)
ax.grid(lw=1, ls='dashed');


ax.errorbar(groups_rates.mean().DFE, 
            groups_rates.mean().Both, 
            yerr=np.array(groups_rates.std().Both),
            #xerr=,
            marker='.', 
            ls='None',
            lw=1.5,
            color='g', 
            ms=1)

trans = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)

rect = patches.Rectangle((0,0), width=mindfe, height=1,
                         transform=trans, color='yellow',
                         alpha=0.5)

ax.add_patch(rect);
ax.set_xlim((xmin,xmax+3));

In [ ]:
import statsmodels.formula.api as smf
mod = smf.ols("Both ~ DFE", rates[selection]).fit()
print(mod.summary())

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
prstd, iv_l, iv_u = wls_prediction_std(mod)

fig = plt.figure()
ax = fig.add_subplot(111)
rates.plot(kind='scatter', x='DFE', y='Both', ax=ax)

xmin, xmax = min(rates['DFE']), max(rates['DFE'])

ax.plot([mindfe, xmax], [mod.fittedvalues.min(), mod.fittedvalues.max()], 
        'IndianRed', lw=1.5)
ax.plot([mindfe, xmax], [iv_u.min(), iv_u.max()], 'r--', lw=1.5)
ax.plot([mindfe, xmax], [iv_l.min(), iv_l.max()], 'r--', lw=1.5)

ax.errorbar(groups_rates.mean().DFE, 
            groups_rates.mean().Both, 
            yerr=np.array(groups_rates.std().Both),
            ls='None',
            lw=1.5,
            color='Green')

trans = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)

rect = patches.Rectangle((0,0), width=mindfe, height=1,
                         transform=trans, color='Yellow',
                         alpha=0.5)

ax.add_patch(rect)
ax.grid(lw=1, ls='dashed')
ax.set_xlim((xmin,xmax+3));


In [ ]:
from pandas.io import data, wb

In [ ]:
wb.search('gdp.*capita.*').iloc[:,:2].head(10)

In [ ]:
dat = wb.download(indicator='NY.GDP.PCAP.PP.CD', country='all', start=2014, end=2014)
#dat = wb.download(indicator='NY.GDP.PCAP.PP.KD', country='all', start=2014, end=2014)
dat.head()

In [ ]:
country = np.array(dat.index.tolist())[:,0]
gdp = np.array(np.array(dat['NY.GDP.PCAP.PP.CD']))
data = pd.DataFrame(data=np.array([country,gdp]).T, columns=['country', 'gdp'])
print(dat['NY.GDP.PCAP.PP.CD'].head())
print(data.head())

In [ ]:
rates['GDP_CD'] = ''
for i in np.arange(len(data)):
    rates.loc[rates.Country.isin([data.country[i]]), ['GDP_CD'] ] = data.loc[i, ['gdp']].values.astype('float')
rates.loc[rates.GDP_CD.isin(['']), ['GDP_CD']] = np.nan

In [ ]:
print(rates[rates.Country=='Sweden'])
print(data[data.country=='Sweden'])
print(data.loc[218, ['gdp']].values.astype('float'))
rates.loc[rates.Country.isin(['Sweden'])]

In [ ]:
selection = ~rates.DFE.isnull() * rates.DFE>mindfe
selection *=  ~rates.GDP_CD.isnull()

In [ ]:
plt.plot(rates[selection].Both.values, rates[selection].GDP_CD.values/1000, '.', ms=10)
plt.ylabel('GDP/1000')
plt.xlabel('Suicide rate');

In [ ]:
plt.scatter(rates[selection].DFE.values, rates[selection].GDP_CD.values/1000, s=rates[selection].Both.values**1.5)
plt.ylabel('GDP/1000')
plt.xlabel('DFE');

In [ ]:
import statsmodels.api as sm

A = rates[selection][['DFE', 'GDP_CD']].astype('float')
A['GDP_CD'] = A['GDP_CD']/1000
b = rates[selection]['Both'].astype('float')
A = sm.add_constant(A)
est = sm.OLS(b, A).fit()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
X, Y = np.meshgrid(np.linspace(A.DFE.min(), A.DFE.max(), 100), 
                       np.linspace(A.GDP_CD.min(), A.GDP_CD.max(), 100))
Z = est.params[0] + est.params[1] * X + est.params[2] * Y


fig = plt.figure(figsize=(7, 5))
ax = Axes3D(fig, azim=-135, elev=15)

surf = ax.plot_surface(X, Y, Z, cmap=plt.cm.RdBu, alpha=0.6, linewidth=0)


resid = b - est.predict(A)
ax.scatter(A.DFE, A.GDP_CD, b,  alpha=1.0)

ax.set_xlabel('DFE')
ax.set_ylabel('GDP_CD/1000')
ax.set_zlabel('Both');

In [ ]:
print(est.summary())

In [ ]:
# now plot the whole distribution, show that it is not really easy. Intersting to identify clusters
# save datafram for next chapter. 
# also, test 3D plot with %matplotlib notebook
# for interactive plots.
%matplotlib notebook
selection2 = ~rates.DFE.isnull()

plt.scatter(rates[selection2].GDP_CD.values/1000, rates[selection2].DFE.values, s=rates[selection2].Both.values**1.5)
plt.xlabel('GDP/1000')
plt.ylabel('DFE');

In [ ]:
data = pd.DataFrame(data=rates[['Country','Both','Male','Female','GDP_CD', 'DFE']][~rates.DFE.isnull()])
data.head()

In [ ]:
TABLE_FILE = 'data_ch4.h5'
data.to_hdf(TABLE_FILE, 'ch4data', mode='w', table=True)

In [ ]:
d2 = pd.read_hdf(TABLE_FILE)
d2.head()

# Logistic regression

In [ ]:
k = 1.
m = -5.

y = lambda x: k*x + m
#p = lambda x: np.exp(k*x+m) / (1+np.exp(k*x+m))
p = lambda x: 1 / (1+np.exp(-1*(k*x+m)))

In [ ]:
xx = np.linspace(0,10)
plt.plot(xx,y(xx), label='linear')
plt.plot(xx,p(xx), label='logistic')
plt.plot([0,abs(m)], [0.5,0.5], dashes=(4,4), color='.7')
plt.plot([abs(m),abs(m)], [-.1,.5], dashes=(4,4), color='.7')

# limits, legends and labels
plt.ylim((-.1,1.1))
plt.legend(loc=2)
plt.ylabel('P')
plt.xlabel('xx');

In [ ]:
studytime = [0,0,1.5,2,2.5,3,3.5,4,4,4,5.5,6,6.5,7,7,8.5,9,9,9,10.5,10.5,12,12,12,12.5,13,14,15,16,18]
passed = [0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1]
data = pd.DataFrame(data=np.array([studytime,passed]).T, columns=['Time', 'Pass'])

In [ ]:
data.Time.hist(bins=6)
plt.xlabel('Time')
plt.ylabel('No. students');

In [ ]:
plt.plot(data.Time, data.Pass,'o', mew=0, ms=7,)
plt.ylim(-.1,1.1)
plt.xlim(-0.2,16.2)
plt.xlabel('Time studied')
plt.ylabel('Pass? (0=no, 1=yes)');

In [ ]:
import statsmodels.api as sm
probfit = sm.Logit(data.Pass, sm.add_constant(data.Time, prepend=True))

In [ ]:
fit_results = probfit.fit()

In [ ]:
print(fit_results.summary())

In [ ]:
logit_pars = fit_results.params
intercept_err, slope_err = np.diag(fit_results.cov_params())**.5
fit_results.cov_params()

In [ ]:
intercept = logit_pars['const']
slope = logit_pars['Time']
print(intercept,slope)

In [ ]:
fit_results.conf_int()

In [ ]:
plt.plot(data.Time, data.Pass,'o', mew=0, ms=7, label='Data')
p = lambda x,k,m: 1 / (1+np.exp(-1*(k*x+m)))
xx = np.linspace(0,data.Time.max())
l1 = plt.plot(xx, p(xx,slope,intercept), label='Fit')
plt.fill_between(xx,
    p(xx,slope+slope_err**2, intercept+intercept_err),
    p(xx,slope-slope_err**2, intercept-intercept_err), 
    alpha=0.15,
    color=l1[0].get_color())
plt.ylim(-.1,1.1)
plt.xlim(-0.2,16.2)
plt.xlabel('Time studied')
plt.ylabel('Pass? (0=no, 1=yes)')
plt.legend(loc=2, numpoints=1);

In [ ]:
target=0.5
x_prob = lambda p,k,m: (np.log(p/(1-p))-m)/k
T_max = x_prob(target, slope-slope_err, intercept-intercept_err)
T_min = x_prob(target, slope+slope_err, intercept+intercept_err)
T_best = x_prob(target, slope, intercept)
print('{0}% sucess rate: {1:.1f} +{2:.1f}/-{3:.1f}'.format(int(target*100),T_best,T_max-T_best,T_best-T_min))